# Library Imports

In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score, precision_score, recall_score, f1_score, log_loss
import pandas as pd
import numpy as np
from aif360.metrics import ClassificationMetric
from aif360.datasets import BinaryLabelDataset
import time
from collections import deque


# Data Load

In [5]:
df = pd.read_parquet('data/nhanes_data_processed.parquet')
df.head()

,Weight,Body mass index,Systolic,Diastolic,Gender,Age,Diabetes,Glycohemoglobin,Cholesterol,High-density lipoprotein (HDL),...,Basophils,Red blood cells,Hemoglobin,Red blood cell width,Platelet count,Mean volume of platelets,Coronary heart disease,Blood related diabetes,Moderate-work,Vigorous-work
4,92.5,29.10,122.0,82.0,Male,597.0,No,5.5,7.21,1.08,...,5.397605e-79,5.13,14.5,13.1,209.0,10.4,No,No,17.0,Yes
6,78.0,29.39,130.0,78.0,Female,712.0,No,5.8,6.34,2.73,...,5.397605e-79,4.60,13.4,14.3,244.0,8.2,No,Yes,3.0,No
9,111.8,30.94,152.0,98.0,Male,518.0,No,5.5,3.62,1.31,...,5.397605e-79,5.00,15.4,13.7,167.0,9.4,No,Don't know,13.0,Don't know
13,75.5,27.33,142.0,56.0,Male,973.0,No,5.8,4.50,1.04,...,5.397605e-79,5.32,16.6,12.4,160.0,9.0,No,No,9.0,Yes
14,81.6,26.68,106.0,68.0,Female,459.0,No,4.6,5.15,1.49,...,5.397605e-79,4.14,13.3,11.9,255.0,7.7,No,No,13.0,Yes


# Data Processing

In [6]:
def process_and_load_data(data = pd.DataFrame, target = str):
    #The input should only be a Pandas DataFrame 
 
    #This creates split datasets for training, testing, and validation
    #Additionally it prepares the input data sets for model fitting and predicting
    X = data.drop(target, axis = 1)
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_val =  scaler.transform(X_val)

    return X_train, X_test, X_val, y_train, y_test, y_val

# Adversarial Model - Testing

### Dummy Data Set

In [3]:
np.random.seed(42)

# Number of rows and columns
num_rows = 1000
num_columns = 41

# Generate random float data for 39 columns
float_data = np.random.rand(num_rows, num_columns - 2)

# Generate two binary columns
binary_data = np.random.randint(0, 2, size=(num_rows, 2))

# Combine into a single dataset
data = np.hstack((float_data, binary_data))

# Create column names
column_names = [f"feature_{i}" for i in range(1, num_columns - 1)]
column_names.append("Gender")
column_names.append("CHD")

# Create DataFrame
df = pd.DataFrame(data, columns=column_names)

In [4]:
df.columns[-1]

'CHD'

### Data Processing

In [7]:
feature_columns = df.columns[:-1]  # All columns except the last two (includes sensitive feature)
sensitive_column = df.columns[-2]  # The third last column as the sensitive feature
label_columns = df.columns[-1]  # The last two columns as labels

# Convert Pandas DataFrame to a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((
    df[feature_columns].values.astype(np.float32),  # Features
    df[sensitive_column].values.reshape(-1, 1).astype(np.int32),  # Sensitive Feature
    df[label_columns].values.astype(np.int32)  # Ensure labels are integers
))
# Define batch size
batch_size = 16  

# Shuffle before batching
buffer_size = len(df)  # Ideally, use the dataset size as the buffer
shuffled_dataset = dataset.shuffle(buffer_size, seed=42).batch(batch_size)

In [2]:
sensitive_column

NameError: name 'sensitive_column' is not defined

In [ ]:
def data_processor(data, batch_size):

    # Define features to standardize (Z-score)
    standardize_features = ["Red blood cells", "Hemoglobin", "Albumin", "Protein"]

    # Define features to normalize (Min-Max)
    normalize_features = ["ALT", "AST", "LDH", "Bilirubin", "Triglycerides", "Glucose"]

    # Categorical Columns
    categorical_cols = ['Diabetes', 'Blood related diabetes', 'Vigorous-work']


    # Data Processing Helper Functions
    scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()
    encoder = OneHotEncoder(sparse=False, drop="first")  


    # Binarize Binary Categorical Columns
    data['Gender'] =  data['Gender'].mask(data['Gender'] == 'Male', 1).mask(data['Gender'] == 'Female', 0)
    data['Coronary heart disease'] = data['Coronary heart disease'].mask(data['Coronary heart disease'] == 'Yes', 1).mask(data['Coronary heart disease'] == 'No', 0)

    # Separate Data to Features and Labels
    X = data.drop('Coronary heart disease', axis = 1)
    y = data['Coronary heart disease']

    
    # Split Data into Train, Validatin, and Test Sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    # One-hot Encoding and Standardization/Normalization of Data
    X_train[standardize_features] = scaler.fit_transform(X_train[standardize_features])
    X_val[standardize_features] = scaler.transform(X_val[standardize_features])
    X_test[standardize_features] = scaler.transform(X_test[standardize_features])

    X_train[normalize_features] = minmax_scaler.fit_transform(X_train[normalize_features])
    X_val[normalize_features] = minmax_scaler.transform(X_val[normalize_features])
    X_test[normalize_features] = minmax_scaler.transform(X_test[normalize_features])

    # Convert to DataFrame with correct column names
    X_train_encoded = encoder.fit_transform(X_train[categorical_cols])
    X_val_encoded = encoder.transform(X_val[categorical_cols])
    X_test_encoded = encoder.transform(X_test[categorical_cols])

    train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(categorical_cols), index=X_train.index)
    val_encoded_df = pd.DataFrame(X_val_encoded, columns=encoder.get_feature_names_out(categorical_cols), index=X_val.index)
    test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(categorical_cols), index=X_test.index)

    # Drop original categorical columns and concatenate the new one-hot encoded features
    X_train = X_train.drop(columns=categorical_cols).reset_index(drop=True)
    X_val = X_val.drop(columns=categorical_cols).reset_index(drop=True)
    X_test = X_test.drop(columns=categorical_cols).reset_index(drop=True)

    X_train = pd.concat([X_train, train_encoded_df], axis=1)
    X_val = pd.concat([X_val, val_encoded_df], axis=1)
    X_test = pd.concat([X_test, test_encoded_df], axis=1)

    # Convert Pandas DataFrame to a TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((
    X_train.to_numpy().astype(np.float32),  # Features
    X_train['Gender'].to_numpy().reshape(-1, 1).astype(np.int32),  # Sensitive Feature
    y_train.to_numpy().values.astype(np.int32)  # Ensure labels are integers
    ))

    # Batch Training Set
    buffer_size = len(X_train)  # Ideally, use the dataset size as the buffer
    batched_dataset = dataset.shuffle(buffer_size, seed=42).batch(batch_size)

    return batched_dataset, X_val, y_val, X_test, y_test



In [59]:
for batch in shuffled_dataset.take(1):  
    features, sensitive_features, labels = batch
    print("Features batch shape:", features.shape)  
    print("Sensitive Feature batch shape:", sensitive_features.shape)  
    print("Labels batch shape:", labels.shape)  

Features batch shape: (16, 40)
Sensitive Feature batch shape: (16, 1)
Labels batch shape: (16,)


In [62]:
for step, (x, z, labels) in enumerate(shuffled_dataset.take(5)):  # Check first 5 batches
    unique_labels = np.unique(labels.numpy())
    print(f"Batch {step} unique labels: {labels}")

Batch 0 unique labels: [1 0 1 1 1 0 1 1 0 1 1 0 1 1 0 1]
Batch 1 unique labels: [0 0 1 1 1 0 0 1 1 0 1 0 0 0 1 1]
Batch 2 unique labels: [1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1]
Batch 3 unique labels: [0 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0]
Batch 4 unique labels: [0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1]


In [14]:
class AdversarialModel(keras.Model):
    def __init__(self, input_dim, sensitive_attr,lambda_tradeoff=0.1, GBT_retrain = 5, epochs = 100):
        super().__init__()

        # Initialize Attributes
        self.lambda_tradeoff = lambda_tradeoff  # Trade-off parameter for adversarial penalty
        self.sensitive_attr = sensitive_attr
        self.epochs = epochs
        self.GBT_retrain = GBT_retrain
   
        # Define the main neural network
        self.dense1 = Dense(32, activation='relu', input_dim = input_dim)
        self.dropout1 = Dropout(0.3)  # Added Dropout layer
        self.dense2 = Dense(16, activation='relu')
        self.output_layer = Dense(1, activation='sigmoid')  # Binary classification
    
        
        # Metrics and optimizer for Main Model
        self.loss_fn = keras.losses.BinaryCrossentropy(name="loss")
        self.optimizer = keras.optimizers.Adam(learning_rate=0.001)
        self.main_acc_metric = keras.metrics.BinaryAccuracy(name="accuracy")

        # Adversarial model (Gradient Boosted Trees)
        self.adversarial_model = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.CLASSIFICATION)

    def call(self, inputs, train = False):
        """Forward pass"""
        x = self.dense1(inputs)
        x = self.dropout1(x, training = train)
        x = self.dense2(x)
        return self.output_layer(x)
    
    @tf.function
    def fit(self, data):

        # Number of Batches
        num_batches = len(data)
       
        for epoch in range(self.epochs):
            
            # Epoch Progress Tracking
            start_time = time.time()
            print(f"\nEpoch {epoch + 1}/{self.epochs}")
            progbar = keras.utils.Progbar(target = num_batches)

            # Track epoch loss
            epoch_loss = 0

            # Adversarial Model is Trained at every K Epochs
            if epoch % self.GBT_retrain == 0:
                y_preds = []
                z_labels = []
              

                # Get Predictions For Most Recent Updated Main Model
                for step, (X_batch_train, z_batch_train, _) in enumerate(data):
                    y_preds.append(self(X_batch_train, train=False))  # No .numpy()
                    z_labels.append(z_batch_train)  # No .numpy()

                # ✅ Convert after exiting `@tf.function`
                y_preds = tf.concat(y_preds, axis=0).numpy()  # Convert to NumPy AFTER
                z_labels = tf.concat(z_labels, axis=0).numpy()

                self.adversarial_model.fit(x=y_preds, y=z_labels)

                # Compute Adversarial Model Loss
                adversarial_model_loss  = self.adversarial_model.make_inspector().evaluation()[2]

           
            for step, (X_batch_train,_, y_batch_train) in enumerate(data):

                with tf.GradientTape() as tape:
                    # Forward pass
                    y_pred = self(X_batch_train, train=True)

                    # Compute Main Model Loss
                    main_model_loss = self.loss_fn(y_batch_train, y_pred)

                    # Compute Combined Loss
                    combined_loss = main_model_loss + (main_model_loss / adversarial_model_loss + 1e-7) - (self.lambda_tradeoff * adversarial_model_loss)

                # Compute gradients
                gradients = tape.gradient(combined_loss, self.trainable_weights)

                # Update weights
                self.optimizer.apply_gradients(list(zip(gradients, self.trainable_weights)))

          
                 # Update training metric.
                self.main_acc_metric.update_state(y_batch_train, y_pred)

                # Track loss for epoch summary
                epoch_loss += combined_loss.numpy()


            # Update Progress Bar per batch
            progbar.update(step + 1, values=[("loss", float(combined_loss)), ("accuracy", float(self.main_acc_metric.result()))])
             
                    
        # Final calculations per epoch
        elapsed_time = time.time() - start_time
        time_per_step = elapsed_time / num_batches * 1e6  # Convert to microseconds
        final_accuracy = self.main_acc_metric.result().numpy()
        final_loss = epoch_loss / num_batches

        # Print final epoch stats
        print(f"\n{num_batches}/{num_batches} - {int(elapsed_time)}s {int(time_per_step)}us/step - accuracy: {final_accuracy:.4f} - loss: {final_loss:.4f}")

        # Reset Accuracy for Next Epoch
        self.main_acc_metric.reset_state()

    def predict(self, X_input, threshold = None, raw_probabilities = None):

    
        if threshold is None:
            threshold = 0.11

        if raw_probabilities is None:
            raw_probabilities = False

        pred_proba = super().predict(X_input)

        zpred_proba = self.adversarial_model.predict(pred_proba)

        if raw_probabilities == True:

            return pred_proba, zpred_proba
        
        else:
             
            binary_preds =  (pred_proba >= threshold).astype(int)
            binary_zpreds = (zpred_proba >= threshold).astype(int)

            return binary_preds, binary_zpreds
    

In [15]:
model = AdversarialModel(41,'Gender')

Use /tmp/tmpvutgogyp as temporary training directory


/home/dsilva/DSC-180-Capstone/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-03 18:53:54.218547: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
2025-02-03 18:53:54.218603: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1850] "goss_beta" set but "sampling_method" not equal to "GOSS".
2025-02-03 18:53:54.218625: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1864] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


In [16]:
model.fit(shuffled_dataset)


Epoch 1/100


InaccessibleTensorError: in user code:

    File "/tmp/ipykernel_372/871302263.py", line 61, in fit  *
        y_preds = tf.concat(y_preds, axis=0).numpy()  # Convert to NumPy AFTER
    File "/home/dsilva/DSC-180-Capstone/.venv/lib/python3.11/site-packages/tensorflow/core/function/capture/capture_container.py", line 144, in capture_by_value
        graph._validate_in_scope(tensor)  # pylint: disable=protected-access

    InaccessibleTensorError: <tf.Tensor 'adversarial_model_3_1/dense_11_1/Sigmoid:0' shape=(None, 1) dtype=float32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
    Please see https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values for more information.
    
    <tf.Tensor 'adversarial_model_3_1/dense_11_1/Sigmoid:0' shape=(None, 1) dtype=float32> was defined here:
        File "<frozen runpy>", line 198, in _run_module_as_main
        File "<frozen runpy>", li

### Another Dummy set for Predict Module

In [146]:

# Number of rows and columns
num_rows = 1000
num_columns = 41

# Generate random float data for 39 columns
float_data = np.random.rand(num_rows, num_columns - 2)

# Generate two binary columns
binary_data = np.random.randint(0, 2, size=(num_rows, 2))

# Combine into a single dataset
data = np.hstack((float_data, binary_data))

# Create column names
column_names = [f"feature_{i}" for i in range(1, num_columns - 1)]
column_names.append("Gender")
column_names.append("CHD")

# Create DataFrame
df_predict = pd.DataFrame(data, columns=column_names)

In [148]:
x_val = df_predict.drop(["CHD"], axis = 1)
y_val = df.CHD
x_val.head(5)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,Gender
0,0.762664,0.108972,0.784587,0.356771,0.992820,0.497898,0.002740,0.936143,0.489978,0.564812,...,0.403190,0.474390,0.497667,0.964042,0.672578,0.739626,0.633534,0.735760,0.127226,0.0
1,0.910774,0.566361,0.747105,0.025832,0.397878,0.476149,0.575739,0.326920,0.179086,0.149810,...,0.244063,0.824258,0.112698,0.260016,0.324769,0.226133,0.657886,0.571461,0.823201,1.0
2,0.995462,0.130282,0.655095,0.439705,0.732180,0.792697,0.527991,0.918847,0.477402,0.784071,...,0.172986,0.657642,0.800218,0.668323,0.913286,0.725450,0.693683,0.245175,0.890300,1.0
3,0.298222,0.794529,0.353065,0.637051,0.130808,0.984601,0.920892,0.167425,0.025938,0.090521,...,0.608200,0.969789,0.790762,0.805609,0.484032,0.115285,0.612453,0.731313,0.953025,1.0
4,0.064734,0.612075,0.946083,0.724822,0.075990,0.373723,0.929611,0.171133,0.115018,0.420616,...,0.686042,0.821443,0.215723,0.528370,0.043151,0.745641,0.889469,0.156075,0.644471,1.0


In [149]:
y_val

0      1.0
1      0.0
2      1.0
3      1.0
4      0.0
      ... 
995    0.0
996    0.0
997    1.0
998    0.0
999    0.0
Name: CHD, Length: 1000, dtype: float64

In [174]:
y_preds, zpreds = model.predict(x_val)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
32/32 [==============================] - 0s 648us/step


In [247]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Adam optimizer
    loss='binary_crossentropy', # Binary Crossentropy loss
    metrics=['accuracy']  # Track accuracy
)

In [248]:
model.evaluate(x_val, y_val)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.9963  


[0.9922277331352234, 0.0, 0.0]

In [250]:
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.9963 
[0.9922277331352234, 0.0, 0.0]
Loss =  0.9922277331352234
Accuracy =  0.0


In [165]:
b

{'accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.5009999871253967>}